# Smartnoise Synth

In [1]:
from lomas_client import Client
from snsynth import Synthesizer # Optional

In [2]:
APP_URL = "http://lomas_server"
USER_NAME = "Dr. Antartica"
DATASET_NAME = "PENGUIN"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

In [3]:
penguin_metadata = client.get_dataset_metadata()
penguin_metadata

{'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'species': {'type': 'string',
   'cardinality': 3,
   'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
  'island': {'type': 'string',
   'cardinality': 3,
   'categories': ['Torgersen', 'Biscoe', 'Dream']},
  'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
  'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
  'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
  'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
  'sex': {'type': 'string',
   'cardinality': 2,
   'categories': ['MALE', 'FEMALE']}}}

In [4]:
Synthesizer.list_synthesizers()

['mwem', 'dpctgan', 'patectgan', 'mst', 'pacsynth', 'dpgan', 'pategan', 'aim']

In [5]:
cost = client.estimate_smartnoise_synth_cost(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = ["bill_length_mm"],
    mul_matrix = [],
    synth_params = {},
    nullable = True,
    table_transformer_style = "gan",
)
cost

{'epsilon_cost': 2.0, 'delta_cost': 0.0001}

In [6]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = ["bill_length_mm"],
    mul_matrix = [],
    synth_params = {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 2
    },
    nullable = True,
    table_transformer_style = "gan",
    dummy = True
)
res

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/usr/local/lib/python3.11/site-p

{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f3daa605650>}

In [9]:
model = res["query_response"]

In [10]:
model.sample(5)

,bill_length_mm
0,30.075561
1,30.046139
2,30.004793
3,30.004920
4,30.006769


In [ ]:
import pandas as pd
from snsynth import Synthesizer
from snsynth.transform import (
    AnonymizationTransformer,
    BinTransformer,
    ChainTransformer,
    #DateTimeTransformer,
    LabelTransformer,
    MinMaxTransformer,
    OneHotEncoder,
)
from snsynth.transform.table import TableTransformer

from typing import Dict, List, Optional

In [ ]:
Synthesizer.list_synthesizers()

In [ ]:
from pydantic import BaseModel, Field
class SmartnoiseSynthModel(BaseModel):
    """Model input for a smarnoise-synth query"""

    dataset_name: str
    synth_name: str
    epsilon: float = 0.0
    delta: float = 0.0
    select_cols: List = []
    synth_params: dict = {}
    mul_matrix: List = []
    nullable: bool = True
    condition: Optional[str] = None
    nb_samples: Optional[int] = None
    table_transformer_style: str = "gan"

In [ ]:
from enum import StrEnum
class SSynthSynthesizer(StrEnum):
    """Synthesizer models for smartnoise synth"""

    # Marginal Synthesizer
    AIM = "aim"
    MWEM = "mwem"
    MST = "mst"
    PAC_SYNTH = "pacsynth"

    # Neural Network Synthesizer
    DP_CTGAN = "dpctgan"
    PATE_CTGAN = "patectgan"
    PATE_GAN = "pategan"  # no documentation
    DP_GAN = "dpgan"  # no documentation

    # Hybrid Synthesizer
    QUAIL = "quail"


class SSynthTableTransStyle(StrEnum):
    """Transformer style for smartnoise synth"""

    GAN = "gan"
    CUBE = "cube"


class SSynthColumnType(StrEnum):
    """Type of columns for SmartnoiseSynth transformer pre-processing"""

    PRIVATE_ID = "private_id"
    CATEGORICAL = "categorical"
    CONTINUOUS = "continuous"
    ORDINAL = "ordinal"
    DATETIME = "datetime"


SSYNTH_PRIVATE_COLUMN = "uuid"
SSYNTH_DEFAULT_NB_SAMPLES = 100

In [ ]:
def _preprocess_data(
        private_data: pd.DataFrame,
        query_json: dict,
    ) -> pd.DataFrame:
        """
        Preprocess the data based on the query parameters.

        Args:
            private_data (pd.DataFrame): Private data to be preprocessed
            query_json (dict): (SmartnoiseSynthModelCost): JSON request object
                select_cols (List[str]): List of columns to select
                mul_matrix (List): Multiplication matrix for columns aggregations

        Returns:
            pd.DataFrame: Preprocessed private data
        """
        if query_json.select_cols:
            try:
                private_data = private_data[query_json.select_cols]
            except KeyError as e:
                raise ValueError(
                    "Error while selecting provided select_cols: " + str(e)
                ) from e

        if query_json.mul_matrix:
            try:
                np_matrix = np.array(query_json.mul_matrix)
                mul_private_data = private_data.to_numpy().dot(np_matrix.T)
                private_data = pd.DataFrame(mul_private_data)
            except ValueError as e:
                raise ValueError(
                    f"Failed to multiply provided mul_matrix: {(str(e))}"
                ) from e
        return private_data

In [ ]:
def _categorize_column(data: dict) -> str:
    """
    Categorize the column based on its metadata.

    Args:
        data (dict): Metadata of the column.

    Returns:
        str: Category of the column.
    """
    match data["type"]:
        case "string" | "boolean":
            return SSynthColumnType.CATEGORICAL
        case "int" | "float":
            if "lower" in data.keys():
                return SSynthColumnType.CONTINUOUS
            if "cardinality" in data.keys():
                return SSynthColumnType.CATEGORICAL
            return SSynthColumnType.ORDINAL
        case "datetime":
            return SSynthColumnType.DATETIME
        case _:
            raise ValueError(
                f"Unknown column type in metadata: {data['type']}"
            )

def _get_column_by_types(
    metadata,
    select_cols: List[str],
) -> Dict[str, List[str]]:
    """
    Sort the column in categories based on their types and metadata

    Args:
        metadata (Metadata): Metadata of the dataset
        select_cols (List[str]): List of columns to select

    Returns:
        Dict[str, List[str]]: Dictionnary of list of columns by categories
    """
    col_categories: Dict[str, List[str]] = {
        SSynthColumnType.CATEGORICAL: [],
        SSynthColumnType.CONTINUOUS: [],
        SSynthColumnType.DATETIME: [],
        SSynthColumnType.ORDINAL: [],
        SSynthColumnType.PRIVATE_ID: [],
    }
    for col_name, data in metadata["columns"].items():
        if select_cols and col_name not in select_cols:
            continue
        
        if "private_id" in data.keys():
            col_categories[SSynthColumnType.PRIVATE_ID].append(col_name)
            continue

        # Sort the column in categories based on their types and metadata
        category = _categorize_column(data)
        col_categories[category].append(col_name)

    return col_categories

def _prepare_data_transformer(
    metadata,
    private_data: pd.DataFrame,
    query_json: dict,
):
    """
    Creates the transformer based on the metadata
    The transformer is used to transform the data before synthesis and then
    reverse the transformation after synthesis.
    See https://docs.smartnoise.org/synth/transforms/index.html for documentation
    See https://github.com/opendp/smartnoise-sdk/blob/main/synth/snsynth/
        transform/type_map.py#L40 for get_transformer() method taken as basis.

    Args:
        metadata (Metadata): Metadata of the dataset
        private_data
        query_json (SmartnoiseSynthModelCost): JSON request object for the query
            select_cols (List[str]): List of columns to select
            nullable (bool): True is the data can have Null values, False otherwise
            table_transformer_style: 'gan' or 'cube'

    Returns:
        table_tranformer (TableTransformer) to pre and post-process the data
    """
    col_categories = _get_column_by_types(metadata, query_json.select_cols)
    style = query_json.table_transformer_style
    nullable = query_json.nullable

    constraints = {}
    for col in col_categories[SSynthColumnType.PRIVATE_ID]:
        constraints[col] = AnonymizationTransformer(SSYNTH_PRIVATE_COLUMN)

    if style == SSynthTableTransStyle.GAN:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = MinMaxTransformer(
                lower=metadata["columns"][col]["lower"],
                upper=metadata["columns"][col]["upper"],
                nullable=nullable,
            )
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    MinMaxTransformer(
                        lower=metadata["columns"][col]["lower"],
                        upper=metadata["columns"][col]["upper"],
                        nullable=nullable
                    ),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
    else:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = BinTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    BinTransformer(bins=20, nullable=nullable),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
    print(constraints)
    return TableTransformer.create(
        data=private_data,
        style=style,
        nullable=nullable,
        constraints=constraints,
    )

## Synthesize data

In [ ]:
query_json_pums = {
    "dataset_name": "PENGUIN",
    "synth_name": "dpctgan",
    "epsilon": 0.1,
    "delta": 0.00001,
    "synth_params": {},
    "select_cols": [],
    "mul_matrix": [],
    "nullable": True,
    "condition": None,
    "nb_samples": None,
    "table_transformer_style": "gan",
}
query_json_pums = SmartnoiseSynthModel.model_validate(query_json_pums)

### PUMS

In [ ]:
pums_private_data = pd.read_csv("https://raw.githubusercontent.com/opendp/smartnoise-sdk/main/datasets/PUMS.csv")
print(pums_private_data.head())
pums_metadata = {
    "columns": {
        "age": {'type': 'int', 'lower': 18, 'upper': 70},
        "sex": {'type': 'boolean'},
        "educ": {'type': 'int', 'cardinality': 14},
        "race": {'type': 'int', 'cardinality': 6},
        "income": {'type': 'float', 'lower': 0.0, 'upper': 500_000},
        "married": {'type': 'boolean'},
    }
}

In [ ]:
pums_private_data = _preprocess_data(pums_private_data, query_json_pums)

In [ ]:
pums_transformer = _prepare_data_transformer(pums_metadata, pums_private_data, query_json_pums)
pums_transformer

In [ ]:
pums_encoded = pums_transformer.fit_transform(pums_private_data, epsilon=0.0)
pums_transformer.odometer.spent

In [ ]:
model = Synthesizer.create(
    synth=query_json_pums.synth_name,
    epsilon=query_json_pums.epsilon,
    verbose=True,
    **query_json_pums.synth_params,
)

In [ ]:
model.fit(
    data=pums_private_data,
    transformer=pums_transformer, #pums_transformer,
    preprocessor_eps=0.0, #0.0,  # will error if not 0.
    #nullable=query_json_pums.nullable,
)

In [ ]:
model

In [ ]:
samples = model.sample(5)
print(samples)

### Penguins

In [ ]:
penguin_private_data = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv", index_col=None)
print(penguin_private_data.head())
penguin_metadata = {
 'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {
   'species': {'type': 'string', 'cardinality': 3,'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
   'island': {'type': 'string', 'cardinality': 3, 'categories': ['Torgersen', 'Biscoe', 'Dream']},
   'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
   'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
   'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
   'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
   'sex': {'type': 'string', 'cardinality': 2,'categories': ['MALE', 'FEMALE']}
  }
}

In [ ]:
query_json_penguin = {
    "dataset_name": "PENGUIN",
    "synth_name": "dpctgan",
    "epsilon": 0.1,
    "delta": 0.00001,
    "select_cols": ["bill_length_mm"],
    "synth_params": {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 50
    },
    "mul_matrix": [],
    "nullable": True,
    "table_transformer_style": "gan",
}
query_json_penguin = SmartnoiseSynthModel.model_validate(query_json_penguin)

In [ ]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    verbose=True,
    **query_json_penguin.synth_params,
)
model

In [ ]:
penguin_private_data = _preprocess_data(penguin_private_data, query_json_penguin)

In [ ]:
penguin_transformer = _prepare_data_transformer(penguin_metadata, penguin_private_data, query_json_penguin)
penguin_transformer.transformers

In [ ]:
penguin_encoded = penguin_transformer.fit_transform(penguin_private_data, epsilon=0.0)
penguin_transformer.odometer.spent

In [ ]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    **query_json_penguin.synth_params,
)

In [ ]:
model.fit(
    data=penguin_private_data,
    transformer=penguin_transformer,
    preprocessor_eps=0.0, #0.0,  # will error if not 0.
    nullable=query_json_penguin.nullable,
)

In [ ]:
samples = model.sample(5)
print(samples)

## Serialise

In [45]:
import pickle
from base64 import b64encode
def serialise_model(model) -> str:
    serialised = b64encode(pickle.dumps(model))
    return serialised.decode("utf-8")

In [57]:
str_model = serialise_model(model)
str_model

In [59]:
b64_model = base64.b64decode(str_model)
b64_model

In [ ]:
new_model = pickle.loads(b64_model)
new_model

In [60]:
samples = new_model.sample(5)
print(samples)

   bill_length_mm
0       46.956818
1       45.232080
2       55.124942
3       50.191850
4       57.324367


In [63]:
import json
j_obj = json.dumps(model)

TypeError: Object of type DPCTGAN is not JSON serializable

In [46]:
import base64
import json
from base64 import b64encode
pickled_model = b64encode(pickle.dumps(model))
depickled_model = base64.b64decode(pickled_model)
json_model = json.loads(depickled_model)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
samples = json_model.sample(100)
print(samples)